In [1]:
import hashlib
import os
import pickle
from urllib.request import urlretrieve

import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile

print("Modules successfully imported")

Modules successfully imported


In [2]:
# This function downloads the file in the given url and store it to the current directory
def download(url, file):
    # Check if the file exist
    if not os.path.isfile(file):
        print('Downloading ' + file + ' ...' )
        urlretrieve(url, file)
        print('Downloading Complete')

# Datasets are provided by udacity
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_train.zip', 'notMNIST_train.zip')
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_test.zip', 'notMNIST_test.zip')

# Make sure that the file is not currepted byt checking the hashes
assert hashlib.md5(open('notMNIST_train.zip', 'rb').read()).hexdigest() == 'c8673b3f28f489e9cdf3a3d74e2ac8fa',\
        'notMNIST_train.zip file is corrupted.  Remove the file and try again.'
assert hashlib.md5(open('notMNIST_test.zip', 'rb').read()).hexdigest() == '5d3c7e653e63471c88df796156a9dfa9',\
        'notMNIST_test.zip file is corrupted.  Remove the file and try again.'
    
print('All files downloaded.')

All files downloaded.


In [8]:
# Given the file uncompress the images and store them in an array, do the same for the labels
def uncompress_features_labels(file):
    features = []
    labels = []
    
    with ZipFile(file) as zipf:
        # this is for showing progressbar
        filenames_pbar = tqdm(zipf.namelist(), unit='files')
        
        for filename in filenames_pbar:
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    feature = np.array(image, dtype=np.float32).flatten()
                label = os.path.split(filename)[1][0]
                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)
        
train_features, train_labels = uncompress_features_labels('notMNIST_train.zip')
test_features, test_labels = uncompress_features_labels('notMNIST_test.zip')


 52%|█████▏    | 110037/210001 [00:26<00:22, 4429.03files/s]


100%|██████████| 10001/10001 [00:03<00:00, 3324.62files/s]

In [9]:
docker_size_limit = 150000
train_features, train_labels = resample(train_features, train_labels, n_samples=docker_size_limit)

# Set flags which will be used later
is_features_normal = False
is_labels_encod = False

print('All features and labels uncompressed.')

All features and labels uncompressed.


In [10]:
# normalize the data with max min scaling. It simple converts images in range 0-255 to 0.1-0.9
def normalize_grayscale(image_data):
    a = 0.1
    b = 0.9
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )


if not is_features_normal:
    train_features = normalize_grayscale(train_features)
    test_features = normalize_grayscale(test_features)
    is_features_normal = True

print('Data normalized')

Data normalized


In [11]:
# doing one hot encoding using the LabelBinarizer in sklearn 
if not is_labels_encod:
    encoder = LabelBinarizer()
    encoder.fit(train_labels)
    train_labels = encoder.transform(train_labels)
    test_labels = encoder.transform(test_labels)
    train_labels = train_labels.astype(np.float32)
    test_labels = test_labels.astype(np.float32)
    is_labels_encod = True

In [12]:
# now split the training data into validation dataset with 0.05% of the data
train_features, valid_features, train_labels, valid_lables = train_test_split(train_features, train_labels, test_size=0.05,
    random_state=832289)


In [13]:
# saving the whole data in pickle file which can be later retrived for learning
pickle_file = 'notMNIST.pickle'
if not os.path.isfile(pickle_file):
    print('creating pickle file')
    
    try:
        with open(pickle_file, 'rb') as pfile:
            pickle.dump(
                {
                    'train_dataset': train_features,
                    'train_labels': train_labels,
                    'valid_dataset': valid_features,
                    'valid_labels': valid_labels,
                    'test_dataset': test_features,
                    'test_labels': test_labels,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

Data cached in pickle file.
